# Twitter authentication
ask professor how to get the results only from twitter.com

In [3]:
import requests
from requests_oauthlib import OAuth1

import pandas as pd
import json

# consumer and access keys and secrets for getting access to Twitter API through your application
consumer_key = '2GEDtzlFMJK6agAMkPQoVTwnl'
consumer_secret = '9TvdpLsvdZDbUrihxDd2LUh02P3moWewdAWqTeupJH90SxPkoi'
access_token = '1241443545975791617-Qy2ioSjn5qmKfHN17bSKV1RhWv19et'
access_secret = 'asmAhYweQTDavRPwrs3FkdJd3557g76rcyksDNGT3b9Nx'

auth = OAuth1(consumer_key, consumer_secret, access_token, access_secret)

# Building a query

In [38]:
def create_url(keyword):
    
    url = 'https://api.twitter.com/1.1/search/tweets.json'

    query_params = {'q': keyword,
                    'count' : 100, 
                    'lang' : "en"   
                    }

    res = requests.get(url, params=query_params, auth=auth)
    return (res.json())

# Creating dataframe with information about the tweets that inclucde names of the stocks from the list

In [5]:
pip install vaderSentiment


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [44]:
def df_from_api(company_list):
  df = pd.DataFrame(columns = ['post', 'keyword', 'date', 'link', "sentiment_score"])

  # to make sure that the tweets we see are connected with APPL as a stock 
  for i in range(len(company_list)):
    company_list[i] = company_list[i] + " AND " + "stock"

  # compund sentiment_score per stock
  compund_sentiment_per_stock = {}


  for i in company_list:
    tweets = create_url(i)
    for tweet in tweets['statuses']:
      if len(tweet["entities"]["user_mentions"])!=0:
        if tweet["text"] in df["post"].values:
          continue
        else:
          stock = i.split(" ")[0]
          sentiment_dict = SentimentIntensityAnalyzer().polarity_scores(tweet["text"])
          url = "https://twitter.com/i/web/status/" + tweet["id_str"]
          row = [tweet["text"], stock, tweet["created_at"], url, sentiment_dict["compound"]]
          df.loc[len(df.index)] = row

    stock_df = df.loc[df['keyword'] == stock]
    compund_sentiment_per_stock[stock] = round(stock_df["sentiment_score"].mean(), 3)
  print(compund_sentiment_per_stock)
  
  return df

data = df_from_api(["APPL", "GOOG", "AMZN", "META", "IBM"])

{'APPL': -0.11, 'GOOG': 0.14, 'AMZN': 0.07, 'META': 0.118, 'IBM': -0.045}


,post,keyword,date,link,sentiment_score
0,@elonmusk I sold all my $APPL today. I don’t w...,APPL,Thu Dec 01 23:11:49 +0000 2022,https://twitter.com/i/web/status/1598454854208...,0.6249
1,@friedmandave @proetrie if APPL was a person i...,APPL,Thu Dec 01 22:51:33 +0000 2022,https://twitter.com/i/web/status/1598449753339...,0.0000
2,@EvasTeslaSPlaid @elonmusk Still buying stock ...,APPL,Thu Dec 01 16:48:22 +0000 2022,https://twitter.com/i/web/status/1598358356833...,0.5859
3,"RT @CryptoCoinCoach: dump Apple stock, show th...",APPL,Thu Dec 01 14:25:19 +0000 2022,https://twitter.com/i/web/status/1598322355419...,-0.7574
4,RT @CryptoCoinCoach: so... @elonmusk VS #Apple...,APPL,Thu Dec 01 02:20:20 +0000 2022,https://twitter.com/i/web/status/1598139908220...,-0.2500
...,...,...,...,...,...
56,Generic blockchains continue to fail enterpris...,IBM,Wed Nov 30 21:03:41 +0000 2022,https://twitter.com/i/web/status/1598060219334...,-0.5423
57,RT @CryptoNewswire: IBM and Australian Stock M...,IBM,Wed Nov 30 20:45:05 +0000 2022,https://twitter.com/i/web/status/1598055540588...,-0.5106
58,RT @CoinDesk: The demise of @IBM’s shipping le...,IBM,Wed Nov 30 20:44:25 +0000 2022,https://twitter.com/i/web/status/1598055369959...,0.0000
59,The demise of @IBM’s shipping ledger TradeLens...,IBM,Wed Nov 30 20:33:53 +0000 2022,https://twitter.com/i/web/status/1598052721344...,0.0000
